In [31]:
import openai
import pandas as pd
import tqdm
import glob

import warnings
warnings.filterwarnings("ignore")

In [32]:
with open("../apikey/apikey.txt", "r") as f:
    openai.api_key = f.readline().replace("\n", "")

In [33]:
def query_prompt(prompt, max_tokens=1000):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=0.3,
    )
    return response["choices"][0]["message"]["content"]

In [34]:
import random

In [37]:
def get_df(fname):
    def f(categories):
        for cat in categories:
            for word in [
                "geology",
                "physics",
                "chemistry",
                "mathematical",
                "biology",
                "astronomy",
                "ecology",
                "genetics",
                "statistics",
                "theoretical"
            ]:
                if word.lower() in cat.lower():
                    return True
            
        return False
    
    def text_preprocess(text):
        return text.replace("====", "\n\n").replace("===", "\n\n").replace("==", "\n\n")

    def sep_n(text, n=80):
        try:
            text = text.split(".")
            return ".".join(text[:n])
        except:
            return None
    
    df = pd.read_parquet(fname)
    df_science = df[df["category"].apply(f)]
    df_science["text"] = df_science["text"].apply(text_preprocess)
    df_science["text"] = df_science["text"].apply(sep_n)
    df_science = df_science[df_science["text"].notnull()]
    return df_science.sample(len(df_science)//5)

In [38]:
files = glob.glob("../data/wikipedia_fixed/*.parquet")

In [39]:
import time

In [40]:
import pickle

In [41]:
import json

In [42]:
from datetime import datetime as dt
import os

In [43]:
texts = []

In [46]:
import traceback 
batch_size = 1

def make_prompt(series):
    prompt = f"""
You are a professor at a science university and are creating a test for your students.
Please create one exam based on the provided text, where you choose the most accurately statement from five options for the question.
The output should json format below:
{{"prompt": <the question text>, "answer" <answer(one of A through E)>, "A": <option A>, "B": <option B>, "C": <option C>, "D": <option D>, "E": <option E>}}

Context:
{series['text']}

Attention:
- The question should be LONG sentences.

"""
    return prompt

def f(series):
    try:
        if series["A"] != series["A"]:
            if type(series["answer"]) == dict:
                for key in ["A", "B", "C", "D", "E"]:
                    series[key] = series["choices"][key]
            elif type(series["answer"] == list):
                for i, key in enumerate(["A", "B", "C", "D", "E"]):
                    series[key] = series["choices"][i]
    except:
        return series
    return series

now_date = dt.now().strftime("%Y%m%d%H%M%S")

first = True
for file in files:
    if os.path.basename(file) in ["all.parquet"]:
        print(f"pass: {file}")
        continue
    df_science = get_df(file)
    
    for i in tqdm.tqdm(range(len(df_science)), desc=file):
        try:
            series = df_science.iloc[i]
            prompt = make_prompt(series)
            text = query_prompt(prompt)
            texts_json = json.loads(text)
            if first:
                print("--------------- text ------------------: ")
                print(series["text"])
                print("--------------- json ------------------: ")
                print(texts_json)
                first = False
            if type(texts_json) == dict:
                texts_json["wiki_id"] = series["id"]
                texts_json["original_text"] = series["text"]
                texts.append(texts_json)
                for col in ["A", "B", "C", "D", "E", "answer", "prompt"]:
                    if col not in texts_json:
                        print(f"{col} not existed: {texts_json}")
            else:
                for text_json in texts_json:
                    text_json["wiki_id"] = series["id"]
                    text_json["original_text"] = series["text"]
                    texts.append(text_json)
                    if col not in text_json:
                        print(f"{col} not existed: {texts_json}")
        except Exception as e:
            print(e)
            traceback.print_exc()
            print(text)
            time.sleep(10)
        if i % 20 == 0:
            df_texts = pd.DataFrame(texts)
            df_texts = df_texts.apply(f, axis=1)

            df_texts.to_parquet(f"output_gpt3.5_generate/{now_date}.parquet")

../data/wikipedia_fixed/a.parquet:   0%|                                           | 1/614 [00:05<59:47,  5.85s/it]

--------------- text ------------------: 
Accelerated solvent extraction (ASE) is a method for extracting various chemicals from a complex solid or semisolid sample matrix. The process uses high temperature and pressure, which results in the extraction taking less time and requiring less solvent, and possibly also giving better analyte recovery, than traditional methods that use less extreme conditions. The elevated temperature is employed to increase extraction efficiency of the analyte of interest and the elevated pressure is used to keep the solvent in a liquid state as the temperature is increased above its boiling point. An automated system for the process was developed by Dionex, a company owned by Thermo Fisher Scientific. The extraction cell is filled with the solid sample to be examined and placed in a temperature-controllable oven. After adding the solvent, the cell is heated at constant pressure (adjustable between 0.3 and 20 MPa) up to a maximum temperature of 200°C and kep

../data/wikipedia_fixed/a.parquet:   5%|█▉                                      | 30/614 [03:34<1:03:04,  6.48s/it]

The server is overloaded or not ready yet.
{"prompt": "What is the theory of animal magnetism?", "answer": "C", "A": "Animal magnetism is a theory that posits the existence of an invisible natural force possessed by all living things.", "B": "Animal magnetism is a theory that claims the force could have physical effects, including healing.", "C": "Animal magnetism is a theory invented by German doctor Franz Mesmer in the 18th century.", "D": "Animal magnetism is a theory that attracted numerous followers in Europe and the United States.", "E": "Animal magnetism is a theory that is no longer practiced today except as a form of alternative medicine in some places."}


Traceback (most recent call last):
  File "/tmp/ipykernel_18466/658324387.py", line 46, in <module>
    text = query_prompt(prompt)
  File "/tmp/ipykernel_18466/2719080132.py", line 2, in query_prompt
    response = openai.ChatCompletion.create(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/abstract/engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 700, in _interpret_response
    self._interpret_response_line(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 745, in _interpret_response_line
    raise error.Serv

...

Traceback (most recent call last):
  File "/tmp/ipykernel_18466/658324387.py", line 46, in <module>
    text = query_prompt(prompt)
  File "/tmp/ipykernel_18466/2719080132.py", line 2, in query_prompt
    response = openai.ChatCompletion.create(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/abstract/engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 700, in _interpret_response
    self._interpret_response_line(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 745, in _interpret_response_line
    raise error.Serv

The server is overloaded or not ready yet.
{"prompt": "Which of the following statements accurately describes the compounds Ytterbium(II) iodide and Ytterbium(III) iodide?", "answer": "C", "A": "Ytterbium(II) iodide is also known as ytterbium diiodide, while Ytterbium(III) iodide is also known as ytterbium triiodide.", "B": "Ytterbium(II) iodide is a compound with the chemical formula YbI3, while Ytterbium(III) iodide is a compound with the chemical formula YbI2.", "C": "Ytterbium(II) iodide is also known as ytterbium diiodide, while Ytterbium(III) iodide is also known as ytterbium triiodide.", "D": "Ytterbium(II) iodide and Ytterbium(III) iodide are both compounds with the chemical formula YbI2.", "E": "Ytterbium(II) iodide and Ytterbium(III) iodide are both compounds with the chemical formula YbI3."}


Traceback (most recent call last):
  File "/tmp/ipykernel_18466/658324387.py", line 46, in <module>
    text = query_prompt(prompt)
  File "/tmp/ipykernel_18466/2719080132.py", line 2, in query_prompt
    response = openai.ChatCompletion.create(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/abstract/engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 700, in _interpret_response
    self._interpret_response_line(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 745, in _interpret_response_line
    raise error.Serv

Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
{"prompt": "Which model is a helix-coil transition model that describes helix-coil transitions of macromolecules?", "answer": "A", "A": "Zimm–Bragg model", "B": "Alpha helix", "C": "Lifson–Roig model", "D": "Random coil", "E": "Statistical mechanics"}


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/conda/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/opt/conda/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/opt/conda/lib/python3.10/http/client.py", line 287, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During handling of the above exception, another exception occurred

In [47]:
df_texts = pd.DataFrame(texts)
df_texts = df_texts.apply(f, axis=1)

df_texts.to_parquet(f"output_gpt3.5_generate/{now_date}.parquet")

In [ ]:
len(df_texts)